In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgbm
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV,KFold, GridSearchCV

from tensorflow.keras import layers, models
import tensorflow as tf

In [5]:
df = pd.read_excel("option_prices_data.xlsx").dropna().reset_index().drop(columns=['index'])
cols = list(df.columns)
cols

['date_fa',
 'symbol',
 'exp_date',
 'close_price',
 'strike_price',
 'equity_price',
 'annual_volatility',
 'riskfree_rate',
 'time_until_exp',
 'BS']

In [6]:
df

,date_fa,symbol,exp_date,close_price,strike_price,equity_price,annual_volatility,riskfree_rate,time_until_exp,BS
0,1400/03/10,ضخود70771,1400/07/07,32,3000,1740,0.444299,0.196389,0.5042,24.239068
1,1400/03/11,ضخود70771,1400/07/07,30,3000,1686,0.448107,0.196389,0.5000,19.314782
2,1400/03/12,ضخود70771,1400/07/07,37,3000,1738,0.462402,0.196389,0.4958,27.025510
3,1400/03/17,ضخود70771,1400/07/07,48,3000,1808,0.479096,0.196389,0.4750,36.700559
4,1400/03/18,ضخود70771,1400/07/07,53,3000,1894,0.498049,0.196389,0.4708,55.503608
...,...,...,...,...,...,...,...,...,...,...
4863,1399/11/19,ضغدر11191,1399/11/26,1003,10000,10990,0.582477,0.297657,0.0292,1157.298735
4864,1399/11/20,ضغدر11191,1399/11/26,1313,10000,11510,0.598842,0.297657,0.0250,1609.719453
4865,1399/11/21,ضغدر11191,1399/11/26,1887,10000,12060,0.601912,0.297657,0.0208,2125.992878
4866,1399/11/25,ضغدر11191,1399/11/26,2482,10000,12770,0.619110,0.297657,0.0042,2782.493790


In [7]:
symbols = df["symbol"].unique()
TRAIN_SIZE = 0.8
train_df = pd.DataFrame()
test_df = pd.DataFrame()

for symbol in symbols:
    section = df[df["symbol"] == symbol].sample(frac=1).copy()
    l = len(section)
    #each section is different for each symbol
    
    train_df = pd.concat([train_df,section.iloc[:int(l*TRAIN_SIZE),:]])
    test_df = pd.concat([test_df,section.iloc[int(l*TRAIN_SIZE):,:]])
    

In [8]:
#player_list = [['M.S.Dhoni', 36, 75, 5428000],
               ['A.B.D Villers', 38, 74, 3428000],
               ['V.Kholi', 31, 70, 8428000],
               ['S.Smith', 34, 80, 4428000],
               ['C.Gayle', 40, 100, 4528000],
               ['J.Root', 33, 72, 7028000],
               ['K.Peterson', 42, 85, 2528000]]
 
# creating a pandas dataframe
#df = pd.DataFrame(player_list, columns=['Name', 'Age', 'Weight', 'Salary'])
 
# data frame before slicing

#df.iloc[5:,:]

IndentationError: unexpected indent (<ipython-input-8-760b9b34f942>, line 2)

In [9]:
train_df

,date_fa,symbol,exp_date,close_price,strike_price,equity_price,annual_volatility,riskfree_rate,time_until_exp,BS
46,1400/06/01,ضخود70771,1400/07/07,35,3000,2294,0.342544,0.196389,0.1583,5.816632
35,1400/05/11,ضخود70771,1400/07/07,59,3000,2074,0.357907,0.196389,0.2542,6.752809
42,1400/05/20,ضخود70771,1400/07/07,26,3000,2042,0.319810,0.196389,0.2167,1.325296
9,1400/03/25,ضخود70771,1400/07/07,53,3000,2083,0.511338,0.196389,0.4417,93.877139
54,1400/06/13,ضخود70771,1400/07/07,13,3000,2253,0.387240,0.196389,0.1083,2.236195
...,...,...,...,...,...,...,...,...,...,...
4829,1399/09/30,ضغدر11191,1399/11/26,3729,10000,12970,0.586683,0.297657,0.2333,3828.271683
4839,1399/10/14,ضغدر11191,1399/11/26,2347,10000,11520,0.522472,0.297657,0.1750,2262.320064
4863,1399/11/19,ضغدر11191,1399/11/26,1003,10000,10990,0.582477,0.297657,0.0292,1157.298735
4861,1399/11/15,ضغدر11191,1399/11/26,900,10000,10990,0.555995,0.297657,0.0458,1247.895348


In [10]:
test_df

,date_fa,symbol,exp_date,close_price,strike_price,equity_price,annual_volatility,riskfree_rate,time_until_exp,BS
64,1400/06/27,ضخود70771,1400/07/07,1,3000,2004,0.434796,0.196389,0.0500,0.001362
15,1400/04/02,ضخود70771,1400/07/07,51,3000,2116,0.529455,0.196389,0.4083,98.504909
31,1400/04/26,ضخود70771,1400/07/07,81,3000,2088,0.385756,0.196389,0.3167,19.969250
4,1400/03/18,ضخود70771,1400/07/07,53,3000,1894,0.498049,0.196389,0.4708,55.503608
20,1400/04/09,ضخود70771,1400/07/07,74,3000,2174,0.506455,0.196389,0.3792,92.583997
...,...,...,...,...,...,...,...,...,...,...
4864,1399/11/20,ضغدر11191,1399/11/26,1313,10000,11510,0.598842,0.297657,0.0250,1609.719453
4827,1399/09/26,ضغدر11191,1399/11/26,3492,10000,12360,0.602526,0.297657,0.2500,3370.322339
4848,1399/10/27,ضغدر11191,1399/11/26,733,10000,9860,0.486159,0.297657,0.1208,768.756860
4857,1399/11/11,ضغدر11191,1399/11/26,2209,10000,11630,0.534467,0.297657,0.0625,1883.558118


In [11]:
train_data = train_df[["strike_price", "equity_price", "annual_volatility", "riskfree_rate", "time_until_exp"]].values
train_labels = train_df["close_price"].values

test_data = test_df[["strike_price", "equity_price", "annual_volatility", "riskfree_rate", "time_until_exp"]].values
test_labels = test_df["close_price"].values

In [12]:
#LightGBM
reg = lgbm.LGBMRegressor(
    # n_estimators=15000,
    # objective= 'mse',
    # learning_rate=0.002,
    # max_depth=32,
    # lambda_l1=30,
    # num_leaves =,
    # max_depth =
    )

param_grid = {
    "n_estimators": list(range(13000,20000,1000)),
    "learning_rate": np.arange(0.0001, 0.0021, 0.0002),
    "num_leaves":  list(range(30, 80, 10)),
    "max_depth": list(range(5, 30,4)),
    # "min_data_in_leaf":  list(range(200, 10000, 100)),
    # "lambda_l1": list(range( 0, 100, 5)),
    "reg_lambda": list(range(50, 100, 10)),
    # "min_gain_to_split": np.arange(0, 10,0.2)
}

lgb_search = RandomizedSearchCV(reg, param_grid,cv=3, n_iter=100, scoring='neg_mean_squared_error', verbose=3)
# eval_set = [(test_data, test_labels)]

In [13]:
reg = lgbm.LGBMRegressor(
    n_estimators=16000,
    objective= 'mse',
    learning_rate=0.0008,
    max_depth=8,
    reg_lambda=80,
    num_leaves =30
)
reg.fit(
    train_data,
    train_labels
)
best_lgb_model = reg

In [14]:
lgbm_pred = best_lgb_model.predict(test_data)

In [15]:
print("LightGBM test RMSE:", mean_squared_error(lgbm_pred, test_df["close_price"].values)**0.5)
print("Black-Sholes test RMSE:", mean_squared_error(test_df["BS"].values, test_df["close_price"].values)**0.5)

LightGBM test RMSE: 381.96077439671456
Black-Sholes test RMSE: 797.9808451315043


In [16]:
#lgbm_pred

In [17]:
#knn
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_X = scaler.fit_transform(train_data)
test_X = scaler.fit_transform(test_data)


In [18]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
model.fit(train_X, train_labels)

KNeighborsRegressor()

In [19]:
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(test_labels, model.predict(test_X))
print("mse = ",mse," & rmse = ", sqrt(mse))

mse =  134856.30686390534  & rmse =  367.2278677659218


In [20]:
#model.predict(test_X)

In [21]:
from sklearn.svm import SVR

In [29]:
#support vector regression
svr_model = SVR(
        C= 305,
        cache_size= 200,
        coef0= 0.0,
        degree= 3,
        epsilon= 0.1,
        gamma= 0.1,
        kernel= 'rbf',
        max_iter= -1,
        shrinking= False,
        tol= 0.001,
        verbose= False
    )
svr_model.fit(
    train_X,
    train_labels
)
best_svr_model = svr_model

In [30]:
svr_pred = best_svr_model.predict(test_X)

In [31]:
print('SVR RMSE:\t\t', mean_squared_error(svr_pred, test_labels)**0.5)
print('Black-Scholes RMSE:\t', mean_squared_error(test_df["BS"], test_labels)**0.5)

SVR RMSE:		 659.7959536223675
Black-Scholes RMSE:	 797.9808451315043
